In [ ]:
import pandas as pd
import numpy as np
from read_house_data # 自作関数 趣旨とは関係ない

from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization

In [ ]:
# データの読み込み
path_train = '../input/train.csv'
path_test = '../input/test.csv'
train, y, test = read_house_data.read_data(path_train, path_test)
X = train.values

In [ ]:
# リッジ回帰で最適化
from sklearn.linear_model import BayesianRidge

# 評価関数の作成
# パラメータはハイパーパラメータ
def validate(alpha_1, alpha_2, lambda_1, lambda_2):
    model = BayesianRidge(alpha_1=alpha_1,
                          alpha_2=alpha_2,
                          lambda_1=lambda_1,
                          lambda_2=lambda_2)
    
    result = -cross_val_score(model, X, y, scorint='neg_mean_squared_error', cv=10) # 評価関数
    result = np.sqrt(result)
    
    return -result.mean() # 最大化をするので、マイナスを付け直す。

# パラメータの領域
pbounds = {
    'alpha_1': (1e-08, 10.),
    'alpha_2': (1e-08, 10.),
    'lambda_1': (1e-08, 10.),
    'lambda_2': (1e-08, 10.)
}

optimizer = BayesianOptimization(f=validate, pbounds=pbounds)

optimizer.maximize(init_points=10, n_iter=10, acq='ucb')

### 計算結果がずらずら出てくる。

# 結果を見る。
print(optimizer.res)

# 最大値を取るパラメータを取得する。
print(optimizer.max)

In [ ]:
# ランダムフォレストの場合
from sklearn.ensemble import RandomForestRegressor

def validate(n_estimators, min_samples_split, max_features):
    model = RandomForestRegressor(n_estimators=int(n_estimators), # 整数しか取らないパラメータにはint()
                                  min_samples_split=int(min_samples_split),
                                  max_features=max_features, random_state=1)
    result = -corss_val_score(model, X, y, scorint='neg_mean_squared_error', cv=10, n_jobs=-1)
    result = np.sqrt(result)
    return -result.mean()

pbounds = {
    'n_estimators': (50, 200),
    'min_samples_split': (2, 25),
    'max_features': (0.1, 0.99)
}

optimizer = BayesianOptimization(f=validate, pbounds=pbounds)
optimizer.maximize(init_points=10, n_iter=20, acq='ucb')

In [ ]:
# ベイズ最適化　xgboost
from xgboost import XGBRegressor

def validate(max_depth, learning_rate, subsample, colsample_bytree, min_child_weight, reg_lambda):    
    model = XGBRegressor(max_depth=int(max_depth),
                         learning_rate=learning_rate,
                         subsample=subsample,
                         colsample_bytree=colsample_bytree,
                         min_child_weight=int(min_child_weight),
                         reg_lmabda=reg_lambda,
                         random_state=1, n_jobs=-1, n_estimators=500)
    
    result = -cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=10)
    result = np.sqrt(result)
    return -result.mean()

pbounds = {
    'max_depth': (5, 15),
    'learning_rate': (0.1, 0.9),
    'subsample': (0.5, 1),
    'colsample_bytree': (0.1, 1),
    'min_child_weight': (1, 20),
    'reg_lambda': (0, 10)
}
    
optimizer = BayesianOptimization(f=validate, pbounds=pbounds)
optimizer.maximize(init_points=10, n_iter=10, acq='ucb')

参考にしたURL:
https://ohke.hateblo.jp/entry/2018/08/04/230000